## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element_...` and when you use `.find_elementSSS_...`

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

In [11]:
driver = webdriver.Chrome()
driver.get("https://webapps1.chicago.gov/buildingrecords/")

In [12]:
driver.find_element_by_xpath("/html/body/div/div[4]/form/div[1]/div[1]/input").click()
driver.find_element_by_id("submit").click()


## Searching

Search for **400 E 41ST ST**.

In [13]:
text_input = driver.find_element_by_id("fullAddress")
text_input.send_keys('400 E 41ST ST')

In [14]:
driver.find_element_by_id("submit").click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

In [15]:
table = driver.find_element_by_id("resultstable_permits")

df = pd.read_html(table.get_attribute('outerHTML'))

In [ ]:
df[0]

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`, but **you also need to save the URL to the inspection**. As a result, you won't be able to use pandas, you'll need to use a loop and create a list of dictionaries.

You can use Selenium (my recommendation) or you can feed the source to BeautifulSoup. You should have approximately 157 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [16]:
inspection_table = driver.find_element_by_xpath('//*[@id="resultstable_inspections"]/tbody')

In [19]:
inspection_rows = inspection_table.find_elements_by_tag_name('tr')
#inspection_href = inspection_rows.get_attribute('innerHTML')
for rows in inspection_rows[:10]:
    print(rows.text)

13175960 11/30/2020 FAILED ANNUAL INSPECTION
12770690 05/30/2019 PASSED BOILER ANNUAL INSPECTION
12670542 05/21/2019 FAILED CONSERVATION ANNUAL
12277260 08/27/2018 FAILED CONSERVATION ANNUAL
12418304 05/30/2018 PASSED BOILER ANNUAL INSPECTION
12136453 06/21/2017 PASSED ANNUAL INSPECTION
12226018 06/21/2017 PASSED ANNUAL INSPECTION
11228963 06/19/2017 FAILED CONSERVATION ANNUAL
12101602 04/21/2017 PASSED ANNUAL INSPECTION
12214968 03/22/2017 PASSED SIGN ANNUAL INSPECTION


In [ ]:
inspection_href = inspection_rows.find_e

### Loopity loops

> If you used Selenium for the last question, copy the code and use it as a starting point for what we're about to do!

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since it opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.